In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer
from datasets import load_dataset, Dataset,DatasetDict
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import pandas as pd
# 1. Load model & tokenizer
model_name = "hasinthakapiyumal/yt-sentiment-analysis-lk"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. Load your test CSV
df = pd.read_csv("test.csv")
dataset = DatasetDict({"test":Dataset.from_pandas(df)})

# 3. Label mapping (edit if different)
label_map = {"negative": 0, "neutral": 1, "positive": 2}

def preprocess(example):
    encoding = tokenizer(example["text"], truncation=True, padding="max_length")
    # encoding["label"] = label_map.get(example["label"], -1)
    return encoding

tokenized_dataset = dataset.map(preprocess)

# 4. Metrics function using sklearn
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# 5. Setup Trainer
trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
)

# 6. Evaluate
eval_results = trainer.evaluate(eval_dataset=tokenized_dataset["test"])

# 7. Show results
for metric, value in eval_results.items():
    if metric.startswith("eval_"):
        print(f"{metric.replace('eval_', '').capitalize()}: {value:.4f}")


Map:   0%|          | 0/249 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hasinthakapiyumal (hasinthakapiyumal-uok) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Loss: 0.9508
Model_preparation_time: 0.0033
Accuracy: 0.7229
Precision: 0.7196
Recall: 0.7229
F1: 0.7193
Runtime: 461.1798
Samples_per_second: 0.5400
Steps_per_second: 0.0690


AttributeError: 'DataFrame' object has no attribute 'unique'

In [ ]:
tokenized_dataset

DatasetDict({
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 249
    })
})